In [1]:
import numpy as np
import pandas as pd
import talib
import datetime as dt
import matplotlib.pyplot as plt
from matplotlib import style
import math
style.use("ggplot")
#plt.style.use('seaborn-white')
plt.rcParams["figure.figsize"] = (20,10)
#plt.rcParams['axes.facecolor'] = 'grey'
#myplot + theme(panel.background = element_rect(fill='green', colour='red'))
import yfinance as yf

In [2]:
current_time = dt.datetime.now()
print(current_time)

2021-03-15 00:05:47.351019


In [3]:
def BollingerBands (df0, period, dev):
    df = df0
    sma = df["Close"].rolling(window = period).mean()
    rstd = df["Close"].rolling(window = period).std()
    df["Standard Moving Average"] = sma
    df["std"] = rstd
    df["Upper Band"] = sma + dev * rstd
    df["Lower Band"] = sma - dev * rstd
    
    return df

In [4]:
def MACD(df0, short, long, signal):
    df = df0
    df["Short Term EMA"] = df["Close"].ewm(span=short, min_periods=short).mean()
    df["Long Term EMA"] = df["Close"].ewm(span=long, min_periods=long).mean()
    df["MACD Line"] = df["Short Term EMA"] - df["Long Term EMA"]
    df["Signal Line"] = df["MACD Line"].ewm(span=signal, min_periods=signal).mean()
    
    return df

In [5]:
def std(df0):
    df = df0
    close_sum = 0
    for i in range (len(df)):
        close_sum += df["Close"].iloc[i]
        
    close_mean = close_sum/len(df)
    variance_sum = 0
    for i in range (len(df)):
            variance_sum += ((close_mean - df["Close"].iloc[i]) * (close_mean - df["Close"].iloc[i]))
    
    variance = variance_sum/len(df)
    std = math.sqrt(variance)
    
    #print(close_sum)
    #print("\n")
    print(close_mean)
    #print("\n")
    print(variance_sum)
    #print("\n")
    #print(variance)
    print(std)

# Back Tester

# Strategy 2_1

In [6]:
stock = yf.Ticker("HINDALCO.NSE").history(start = "2018-01-15", end = "2019-12-01", interval = "1d")  #28-01-2015 ---> 28-02-2015
stock

- HINDALCO.NSE: No data found, symbol may be delisted


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,


In [7]:
dff = BollingerBands(stock, 13, 1.817) + MACD(stock, 7,42, 5)
dff['RSI_14'] = talib.RSI(dff['Close'], timeperiod=14)
df_test = dff.tail(400)
dff.tail(400)

Exception: inputs are all NaN

In [ ]:
#df_test = dff.tail(23)
tradebook = []
long_short = []
timestamp = []
count = 0
npl = 0
position_squared = True
target = 0
stoploss = 0
tradetime = "open"
# long/short info
# price at which bought/shorted
# date at which bought/shorted
# buy/sell info

# 1 for long, 0 for short

days_till_trade = 0

for i in range(len(df_test)-1):
    #print(openp)
    tradetime = "open"
    open_prev = df_test["Open"].iloc[i]
    openp = df_test["Open"].iloc[i+1]
    close = df_test["Close"].iloc[i]
    closep = df_test["Close"].iloc[i+1]
    #p = df_test["Pivot"].iloc[i+1]
    #r1 = df_test["R1"].iloc[i+1]
    #r2 = df_test["R2"].iloc[i+1]
    #r3 = df_test["R3"].iloc[i+1]
    #s1 = df_test["S1"].iloc[i+1]
    #s2 = df_test["S2"].iloc[i+1]
    #s3 = df_test["S3"].iloc[i+1]
    ub = df_test["Upper Band"].iloc[i]
    lb = df_test["Lower Band"].iloc[i]
    sma = df_test["Standard Moving Average"].iloc[i]
    std = df_test["std"].iloc[i]
    
    sema = df_test["Short Term EMA"].iloc[i-1]
    semap = df_test["Short Term EMA"].iloc[i]
    lema = df_test["Long Term EMA"].iloc[i-1]
    lemap = df_test["Long Term EMA"].iloc[i]
    
    if(position_squared == True and i < len(df_test)-2):
            #ub - openp < 0.117*std or       openp - lb < 0.117*std or
        if(((openp > ub) or (ub - openp < 0.097*std)) and tradetime == "open"):
            long_short.append(0)
            tradebook.append(openp)
            timestamp.append(df_test.index[i])
            npl += openp
            count += 1
            position_squared = False
            days_till_trade = 0

            print(npl)
            print("entered short\n")
        elif(((lb > openp) or (openp - lb < 0.097*std)) and tradetime == "open"):
            long_short.append(1)
            tradebook.append(openp)
            timestamp.append(df_test.index[i])
            npl -= openp
            count += 1
            position_squared = False
            days_till_trade = 0
            #tradetime = "close"

            print(npl)
            print("entered long\n")
    elif(position_squared == False):
        if((openp >= tradebook[count-1]) and long_short[count-1] == 1 and tradetime == "open"):
            long_short.append(0)
            tradebook.append(openp)
            timestamp.append(df_test.index[i])
            npl += openp
            count += 1
            position_squared = True
            print(npl)
            print("sold off and squared\n")

        elif((openp <= tradebook[count-1]) and long_short[count-1] == 0 and tradetime == "open"):
            long_short.append(1)
            tradebook.append(openp)
            timestamp.append(df_test.index[i])
            npl -= openp 
            count += 1
            position_squared = True
            print(npl)
            print("bought back and squared\n")

            
    
    tradetime = "close"
            
            
            
    if(position_squared == True and i < len(df_test)-2):
        if(((closep > ub) or (ub - closep < 0.097*std)) and tradetime == "close"):
            long_short.append(0)
            tradebook.append(closep)
            timestamp.append(df_test.index[i])
            npl += closep
            count += 1
            position_squared = False
            days_till_trade = 0
            #tradetime = "open"

            print(npl)
            print("entered short\n")
        elif(((lb > closep) or (closep - lb < 0.097*std)) and tradetime == "close"):
            long_short.append(1)
            tradebook.append(closep)
            timestamp.append(df_test.index[i])
            npl -= closep
            count += 1
            position_squared = False
            days_till_trade = 0
            #tradetime = "open"

            print(npl)
            print("entered long\n")
        
    elif(position_squared == False):
        if((closep >= tradebook[count-1]) and long_short[count-1] == 1 and tradetime == "close"):
            long_short.append(0)
            tradebook.append(closep)
            timestamp.append(df_test.index[i])
            npl += closep
            count += 1
            position_squared = True
            print(npl)
            print("sold off and squared\n")

        elif((closep <= tradebook[count-1]) and long_short[count-1] == 0 and tradetime == "close"):
            long_short.append(1)
            tradebook.append(closep)
            timestamp.append(df_test.index[i])
            npl -= closep 
            count += 1
            position_squared = True
            print(npl)
            print("bought back and squared\n")
        
        
        
    tradetime = "open"
    days_till_trade += 1
        
    if(i == len(df_test) - 2 and position_squared == False):           
        if(long_short[count-1] == 1):
            long_short.append(0)
            tradebook.append(closep)
            timestamp.append(df_test.index[i])
            npl += closep
            count += 1
            position_squared = True

            print(npl)
            print("sold off and squared\n")

        elif(long_short[count-1] == 0):
            long_short.append(1)
            tradebook.append(closep)
            timestamp.append(df_test.index[i])
            npl -= closep 
            count += 1
            position_squared = True 

            print(npl)
            print("bought back and squared\n")
    
    
    
        
print(npl)
print("\n")
print(tradebook)
print("\n")
print(timestamp)

In [ ]:
print(tradebook)
print("\n")
print(timestamp)
print("\n")
print(npl)

# #Ignore the code below

# Strategy2_2

In [10]:
#df_test = dff.tail(23)
tradebook = []
long_short = []
timestamp = []
count = 0
npl = 0
position_squared = True
target = 0
stoploss = 0
tradetime = "open"
# long/short info
# price at which bought/shorted
# date at which bought/shorted
# buy/sell info

# 1 for long, 0 for short

days_till_trade = 0

for i in range(len(df_test)-1):
    #print(openp)
    tradetime = "open"
    open_prev = df_test["Open"].iloc[i]
    openp = df_test["Open"].iloc[i+1]
    close = df_test["Close"].iloc[i]
    closep = df_test["Close"].iloc[i+1]
    #p = df_test["Pivot"].iloc[i+1]
    #r1 = df_test["R1"].iloc[i+1]
    #r2 = df_test["R2"].iloc[i+1]
    #r3 = df_test["R3"].iloc[i+1]
    #s1 = df_test["S1"].iloc[i+1]
    #s2 = df_test["S2"].iloc[i+1]
    #s3 = df_test["S3"].iloc[i+1]
    ub = df_test["Upper Band"].iloc[i]
    lb = df_test["Lower Band"].iloc[i]
    sma = df_test["Standard Moving Average"].iloc[i]
    std = df_test["std"].iloc[i]
    
    sema = df_test["Short Term EMA"].iloc[i-1]
    semap = df_test["Short Term EMA"].iloc[i]
    lema = df_test["Long Term EMA"].iloc[i-1]
    lemap = df_test["Long Term EMA"].iloc[i]
    
    if(position_squared == True and i < len(df_test)-2):
            #ub - openp < 0.117*std or       openp - lb < 0.117*std or
        if(((openp > ub) or (ub - openp < 0.097*std)) and tradetime == "open"):
            long_short.append(0)
            tradebook.append(openp)
            timestamp.append(df_test.index[i])
            npl += openp
            count += 1
            position_squared = False
            days_till_trade = 0

            print(npl)
            print("entered short\n")
        elif(((lb > openp) or (openp - lb < 0.097*std)) and tradetime == "open"):
            long_short.append(1)
            tradebook.append(openp)
            timestamp.append(df_test.index[i])
            npl -= openp
            count += 1
            position_squared = False
            days_till_trade = 0
            #tradetime = "close"

            print(npl)
            print("entered long\n")
    elif(position_squared == False):
        if((openp >= tradebook[count-1] or (days_till_trade >= 7 and npl+openp > (npl+tradebook[count-1])*0.25)) and long_short[count-1] == 1 and tradetime == "open"):
            long_short.append(0)
            tradebook.append(openp)
            timestamp.append(df_test.index[i])
            npl += openp
            count += 1
            position_squared = True
            print(npl)
            print("sold off and squared\n")

        elif((openp <= tradebook[count-1] or (days_till_trade >= 7 and npl-openp > (npl-tradebook[count-1])*0.25)) and long_short[count-1] == 0 and tradetime == "open"):
            long_short.append(1)
            tradebook.append(openp)
            timestamp.append(df_test.index[i])
            npl -= openp 
            count += 1
            position_squared = True
            print(npl)
            print("bought back and squared\n")

            
    
    tradetime = "close"
            
            
            
    if(position_squared == True and i < len(df_test)-2):
        if(((closep > ub) or (ub - closep < 0.097*std)) and tradetime == "close"):
            long_short.append(0)
            tradebook.append(closep)
            timestamp.append(df_test.index[i])
            npl += closep
            count += 1
            position_squared = False
            days_till_trade = 0
            #tradetime = "open"

            print(npl)
            print("entered short\n")
        elif(((lb > closep) or (closep - lb < 0.097*std)) and tradetime == "close"):
            long_short.append(1)
            tradebook.append(closep)
            timestamp.append(df_test.index[i])
            npl -= closep
            count += 1
            position_squared = False
            days_till_trade = 0
            #tradetime = "open"

            print(npl)
            print("entered long\n")
        
    elif(position_squared == False):
        if((closep >= tradebook[count-1] or (days_till_trade >= 7 and npl+openp > (npl+tradebook[count-1])*0.25)) and long_short[count-1] == 1 and tradetime == "close"):
            long_short.append(0)
            tradebook.append(closep)
            timestamp.append(df_test.index[i])
            npl += closep
            count += 1
            position_squared = True
            print(npl)
            print("sold off and squared\n")

        elif((closep <= tradebook[count-1] or (days_till_trade >= 7 and npl-openp > (npl-tradebook[count-1])*0.25)) and long_short[count-1] == 0 and tradetime == "close"):
            long_short.append(1)
            tradebook.append(closep)
            timestamp.append(df_test.index[i])
            npl -= closep 
            count += 1
            position_squared = True
            print(npl)
            print("bought back and squared\n")
        
        
        
    tradetime = "open"
    days_till_trade += 1
        
    if(i == len(df_test) - 2 and position_squared == False):           
        if(long_short[count-1] == 1):
            long_short.append(0)
            tradebook.append(closep)
            timestamp.append(df_test.index[i])
            npl += closep
            count += 1
            position_squared = True

            print(npl)
            print("sold off and squared\n")

        elif(long_short[count-1] == 0):
            long_short.append(1)
            tradebook.append(closep)
            timestamp.append(df_test.index[i])
            npl -= closep 
            count += 1
            position_squared = True 

            print(npl)
            print("bought back and squared\n")
    
    
    
        
print(npl)
print("\n")
print(tradebook)
print("\n")
print(timestamp)

296.82611083984375
entered short

1.055908203125
bought back and squared

-293.6968688964844
entered long

3.4747896268719956
sold off and squared

304.75460883040915
entered short

1.4590897412229538
bought back and squared

-298.3233504443239
entered long

2.169361373017921
sold off and squared

-298.9568593301071
entered long

2.7452539552527924
sold off and squared

-295.4629674802941
entered long

12.631839196453427
sold off and squared

-304.4472013308903
entered long

12.996585858760284
sold off and squared

-298.0353660455366
entered long

13.418950318175405
sold off and squared

-292.62176013104335
entered long

15.108284012814295
sold off and squared

-290.7596359090607
entered long

16.98958656002486
sold off and squared

-289.60782554935014
entered long

17.834221569790486
sold off and squared

328.44385889139136
entered short

8.274035748452661
bought back and squared

328.6166260804839
entered short

5.5673001646803755
bought back and squared

335.892403924446
entered sho

In [11]:
print(tradebook)
print("\n")
print(timestamp)
print("\n")
print(npl)

[296.82611083984375, 295.77020263671875, 294.7527770996094, 297.17165852335637, 301.27981920353716, 303.2955190891862, 299.7824401855469, 300.49271181734184, 301.126220703125, 301.70211328535987, 298.2082214355469, 308.0948066767475, 317.07904052734375, 317.4437871896506, 311.0319519042969, 311.454316363712, 306.04071044921875, 307.73004414385764, 305.867919921875, 307.74922246908557, 306.597412109375, 307.4420471191406, 310.60963732160087, 320.1698231429387, 320.34259033203125, 323.04932591580354, 330.3251037597656, 329.6915557910675, 329.49957275390625, 329.1348066900708, 330.9777526855469, 324.2971667238086, 326.15924072265625, 317.768633583467, 319.9383544921875, 318.7178876396247, 304.1109313964844, 307.13306203071636, 321.5269322530549, 318.1173400878906, 299.75210504884666, 290.5889020477001, 293.64978324790445, 291.9836723179442, 294.9283142089844, 294.5021667480469, 298.51226806640625, 297.36925662619484, 299.7715148925781, 298.6285241127637, 308.1791800402395, 307.26869544891

In [12]:
print(std(df_test))

TypeError: 'numpy.float64' object is not callable

In [329]:
closep > tradebook[count-1]*1.0007 and 
closep < tradebook[count-1]*0.9993 and 

 and semap - sema > lemap - lema and semap > lemap
and semap - sema < lemap - lema and semap < lemap

SyntaxError: invalid syntax (<ipython-input-329-ee1377b10203>, line 1)